# Exploración de Datos

In [50]:
import pandas as pd

In [51]:
data = pd.read_csv("../data/Bank_Transaction_Fraud_Detection.csv")

## Información de Datos

In [52]:
data.head(n=10)

,Customer_ID,Customer_Name,Gender,Age,State,City,Bank_Branch,Account_Type,Transaction_ID,Transaction_Date,...,Merchant_Category,Account_Balance,Transaction_Device,Transaction_Location,Device_Type,Is_Fraud,Transaction_Currency,Customer_Contact,Transaction_Description,Customer_Email
0,d5f6ec07-d69e-4f47-b9b4-7c58ff17c19e,Osha Tella,Male,60,Kerala,Thiruvananthapuram,Thiruvananthapuram Branch,Savings,4fa3208f-9e23-42dc-b330-844829d0c12c,23-01-2025,...,Restaurant,74557.27,Voice Assistant,"Thiruvananthapuram, Kerala",POS,0,INR,+9198579XXXXXX,Bitcoin transaction,oshaXXXXX@XXXXX.com
1,7c14ad51-781a-4db9-b7bd-67439c175262,Hredhaan Khosla,Female,51,Maharashtra,Nashik,Nashik Branch,Business,c9de0c06-2c4c-40a9-97ed-3c7b8f97c79c,11-01-2025,...,Restaurant,74622.66,POS Mobile Device,"Nashik, Maharashtra",Desktop,0,INR,+9191074XXXXXX,Grocery delivery,hredhaanXXXX@XXXXXX.com
2,3a73a0e5-d4da-45aa-85f3-528413900a35,Ekani Nazareth,Male,20,Bihar,Bhagalpur,Bhagalpur Branch,Savings,e41c55f9-c016-4ff3-872b-cae72467c75c,25-01-2025,...,Groceries,66817.99,ATM,"Bhagalpur, Bihar",Desktop,0,INR,+9197745XXXXXX,Mutual fund investment,ekaniXXX@XXXXXX.com
3,7902f4ef-9050-4a79-857d-9c2ea3181940,Yamini Ramachandran,Female,57,Tamil Nadu,Chennai,Chennai Branch,Business,7f7ee11b-ff2c-45a3-802a-49bc47c02ecb,19-01-2025,...,Entertainment,58177.08,POS Mobile App,"Chennai, Tamil Nadu",Mobile,0,INR,+9195889XXXXXX,Food delivery,yaminiXXXXX@XXXXXXX.com
4,3a4bba70-d9a9-4c5f-8b92-1735fd8c19e9,Kritika Rege,Female,43,Punjab,Amritsar,Amritsar Branch,Savings,f8e6ac6f-81a1-4985-bf12-f60967d852ef,30-01-2025,...,Entertainment,16108.56,Virtual Card,"Amritsar, Punjab",Mobile,0,INR,+9195316XXXXXX,Debt repayment,kritikaXXXX@XXXXXX.com
5,6c870d65-76b0-431d-bdf3-9292998e8211,Ishanvi Dar,Male,54,Gujarat,Ahmedabad,Ahmedabad Branch,Checking,af5f667c-d064-4083-bfb7-83396111a3da,25-01-2025,...,Entertainment,61258.85,Mobile Device,"Ahmedabad, Gujarat",POS,0,INR,+9198318XXXXXX,Seminar registration,ishanviXXX@XXXXX.com
6,5323737c-bbd2-423f-9c9b-e0433c8f75dc,Arya Shroff,Female,61,Delhi,New Delhi,New Delhi Branch,Business,b1355810-d246-4aeb-9932-347f32646172,04-01-2025,...,Health,36313.61,Payment Gateway Device,"New Delhi, Delhi",Desktop,0,INR,+9194785XXXXXX,Public transport pass,aryaXXX@XXXXX.com
7,c0c3d474-f6c2-4c66-9b0e-f9ba75c6f310,Jackson Shere,Male,32,Andaman and Nicobar Islands,Port Blair,Port Blair Branch,Business,c86a000c-d81f-40be-acdf-77fc072fd808,16-01-2025,...,Clothing,16948.73,Debit/Credit Card,"Port Blair, Andaman and Nicobar Islands",ATM,0,INR,+9193423XXXXXX,Online shopping,jacksonXXX@XXXXXXX.com
8,e9a82764-1253-4a46-ad34-80e3416fc801,Bhanumati Ravel,Male,52,Madhya Pradesh,Bhopal,Bhopal Branch,Business,323cc683-b0dc-40ee-a519-3b5dc96c7ed8,25-01-2025,...,Clothing,18138.71,ATM,"Bhopal, Madhya Pradesh",Mobile,0,INR,+9194374XXXXXX,Vacation payment,bhanumatiXXXXX@XXXXX.com
9,708224d5-192a-4d86-b411-8ec6d1bb274b,Meera Ganesh,Female,32,Chhattisgarh,Jagdalpur,Jagdalpur Branch,Business,9fad31ea-2770-4d80-a0ea-00972d5f02cc,02-01-2025,...,Entertainment,65801.35,Bank Branch,"Jagdalpur, Chhattisgarh",Desktop,0,INR,+9194511XXXXXX,Electronics purchase,meeraXXXXX@XXXXXXX.com


In [53]:
print(f"El dataset contiene {data.shape[0]} filas y {data.shape[1]} columnas.")

El dataset contiene 200000 filas y 24 columnas.


## Valores Duplicados y Nulos

In [54]:
nulos = data.isnull().sum()
flag_nulos = nulos.sum() > 0

duplicados = data.duplicated()
flag_duplicados = duplicados.sum() > 0

print(f"Existen {nulos.sum()} valores nulos.")
if flag_nulos:
  print("Vista previa de valores nulos:\n", nulos[nulos > 0])

print(f"Existen {duplicados.sum()} valores duplicados.")
if flag_duplicados:
  print("Vista previa de valores duplicados:\n", data[duplicados])

Existen 0 valores nulos.
Existen 0 valores duplicados.


## Exploración de Atributos/Columnas

In [55]:
count = 1
for column in data.columns:
  print(f"{count}. {column}")
  count +=1

1. Customer_ID
2. Customer_Name
3. Gender
4. Age
5. State
6. City
7. Bank_Branch
8. Account_Type
9. Transaction_ID
10. Transaction_Date
11. Transaction_Time
12. Transaction_Amount
13. Merchant_ID
14. Transaction_Type
15. Merchant_Category
16. Account_Balance
17. Transaction_Device
18. Transaction_Location
19. Device_Type
20. Is_Fraud
21. Transaction_Currency
22. Customer_Contact
23. Transaction_Description
24. Customer_Email


In [56]:
data.dtypes

Customer_ID                 object
Customer_Name               object
Gender                      object
Age                          int64
State                       object
City                        object
Bank_Branch                 object
Account_Type                object
Transaction_ID              object
Transaction_Date            object
Transaction_Time            object
Transaction_Amount         float64
Merchant_ID                 object
Transaction_Type            object
Merchant_Category           object
Account_Balance            float64
Transaction_Device          object
Transaction_Location        object
Device_Type                 object
Is_Fraud                     int64
Transaction_Currency        object
Customer_Contact            object
Transaction_Description     object
Customer_Email              object
dtype: object

In [66]:
data["Device_Type"].nunique()
data["Device_Type"].unique()
# Definir el mapeo de Device_Type a Device_Trust_Score
trust_scores = {
    "ATM": 0.85,
    "POS": 0.65,
    "Mobile": 0.80,
    "Desktop": 0.70
}

# Crear la nueva columna basado en Device_Type
data["Device_Trust_Score"] = data["Device_Type"].map(trust_scores)

data.head()

,Customer_ID,Customer_Name,Gender,Age,State,City,Bank_Branch,Account_Type,Transaction_ID,Transaction_Date,...,Account_ID,Open_Date,Status,Merchant_Name,Merchant_Location,Risk_Level,Device_ID,Device_Location,Is_Compromised,Device_Trust_Score
0,d5f6ec07-d69e-4f47-b9b4-7c58ff17c19e,Osha Tella,Male,60,Kerala,Thiruvananthapuram,Thiruvananthapuram Branch,Savings,4fa3208f-9e23-42dc-b330-844829d0c12c,2025-01-23,...,None,None,None,None,None,None,None,None,None,0.65
1,7c14ad51-781a-4db9-b7bd-67439c175262,Hredhaan Khosla,Female,51,Maharashtra,Nashik,Nashik Branch,Business,c9de0c06-2c4c-40a9-97ed-3c7b8f97c79c,2025-01-11,...,None,None,None,None,None,None,None,None,None,0.70
2,3a73a0e5-d4da-45aa-85f3-528413900a35,Ekani Nazareth,Male,20,Bihar,Bhagalpur,Bhagalpur Branch,Savings,e41c55f9-c016-4ff3-872b-cae72467c75c,2025-01-25,...,None,None,None,None,None,None,None,None,None,0.70
3,7902f4ef-9050-4a79-857d-9c2ea3181940,Yamini Ramachandran,Female,57,Tamil Nadu,Chennai,Chennai Branch,Business,7f7ee11b-ff2c-45a3-802a-49bc47c02ecb,2025-01-19,...,None,None,None,None,None,None,None,None,None,0.80
4,3a4bba70-d9a9-4c5f-8b92-1735fd8c19e9,Kritika Rege,Female,43,Punjab,Amritsar,Amritsar Branch,Savings,f8e6ac6f-81a1-4985-bf12-f60967d852ef,2025-01-30,...,None,None,None,None,None,None,None,None,None,0.80


In [58]:
df = data.copy()

# Eliminar " Branch" de la columna Bank_Branch
df['Branch_Clean'] = df['Bank_Branch'].str.replace(' Branch$', '', regex=True)

# Filtrar índices donde City y Branch_Clean no coinciden
indices_diferentes = df[df['City'] != df['Branch_Clean']].index.tolist()

print("Índices con diferencias:", indices_diferentes)

Índices con diferencias: []


In [59]:
# Renombrar la columna Is_Fraud y convertir valores de 0/1 a booleano (False/True)
data.rename(columns={"Is_Fraud": "Is_Fraud_Teoric"}, inplace=True)
data["Is_Fraud_Teoric"] = data["Is_Fraud_Teoric"].astype(bool)

## Edición de Datos

In [60]:
# Convertir Transaction_Date al formato YYYY-MM-DD
data["Transaction_Date"] = pd.to_datetime(data["Transaction_Date"], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")

# Verificar que la conversión sea correcta
print(data[["Transaction_Date", "Transaction_Time"]].head(10))

  Transaction_Date Transaction_Time
0       2025-01-23         16:04:07
1       2025-01-11         17:14:53
2       2025-01-25         03:09:52
3       2025-01-19         12:27:02
4       2025-01-30         18:30:46
5       2025-01-25         06:49:53
6       2025-01-04         00:53:33
7       2025-01-16         04:04:48
8       2025-01-25         10:50:12
9       2025-01-02         04:37:34


## Definir Nuevas Variables

In [61]:
# Definir las nuevas columnas y llenarlas con None (equivalente a NULL en bases de datos)
new_columns = {
    "Fraud_Score": None,             # Float (Puntaje de riesgo de fraude)
    "Account_ID": None,              # String (Identificador de la cuenta bancaria)
    "Open_Date": None,               # Date (Fecha de apertura de la cuenta)
    "Status": None,                  # String (Estado de la cuenta)
    "Merchant_Name": None,           # String (Nombre del comerciante)
    "Merchant_Location": None,       # String (Ubicación del comerciante)
    "Risk_Level": None,              # Int (Nivel de riesgo del comerciante)
    "Device_ID": None,               # String (Identificador del dispositivo)
    "Device_Location": None,         # String (Ubicación del dispositivo)
    "Is_Compromised": None           # Boolean (Si el dispositivo ha sido comprometido)
}

# Agregar las nuevas columnas al DataFrame
for col, value in new_columns.items():
  data[col] = value

In [62]:
data.head()


,Customer_ID,Customer_Name,Gender,Age,State,City,Bank_Branch,Account_Type,Transaction_ID,Transaction_Date,...,Fraud_Score,Account_ID,Open_Date,Status,Merchant_Name,Merchant_Location,Risk_Level,Device_ID,Device_Location,Is_Compromised
0,d5f6ec07-d69e-4f47-b9b4-7c58ff17c19e,Osha Tella,Male,60,Kerala,Thiruvananthapuram,Thiruvananthapuram Branch,Savings,4fa3208f-9e23-42dc-b330-844829d0c12c,2025-01-23,...,None,None,None,None,None,None,None,None,None,None
1,7c14ad51-781a-4db9-b7bd-67439c175262,Hredhaan Khosla,Female,51,Maharashtra,Nashik,Nashik Branch,Business,c9de0c06-2c4c-40a9-97ed-3c7b8f97c79c,2025-01-11,...,None,None,None,None,None,None,None,None,None,None
2,3a73a0e5-d4da-45aa-85f3-528413900a35,Ekani Nazareth,Male,20,Bihar,Bhagalpur,Bhagalpur Branch,Savings,e41c55f9-c016-4ff3-872b-cae72467c75c,2025-01-25,...,None,None,None,None,None,None,None,None,None,None
3,7902f4ef-9050-4a79-857d-9c2ea3181940,Yamini Ramachandran,Female,57,Tamil Nadu,Chennai,Chennai Branch,Business,7f7ee11b-ff2c-45a3-802a-49bc47c02ecb,2025-01-19,...,None,None,None,None,None,None,None,None,None,None
4,3a4bba70-d9a9-4c5f-8b92-1735fd8c19e9,Kritika Rege,Female,43,Punjab,Amritsar,Amritsar Branch,Savings,f8e6ac6f-81a1-4985-bf12-f60967d852ef,2025-01-30,...,None,None,None,None,None,None,None,None,None,None


In [63]:
print(f"El dataset contiene {data.shape[0]} filas y {data.shape[1]} columnas.")

El dataset contiene 200000 filas y 34 columnas.


In [64]:
# Guardar el dataset con los cambios en un nuevo archivo CSV
data.to_csv("../data/data_clean.csv", index=False)

# Verificar que se ha guardado correctamente
print("Archivo guardado correctamente: data_clean.csv")

Archivo guardado correctamente: data_clean.csv
